In [2]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import random
import pandas as pd
pd.set_option('display.max_columns', None)
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [4]:
dataset = OnlineCatalog('adult')

In [5]:
dataset.encoder.get_feature_names(dataset.categorical)

array(['marital-status_Non-Married', 'native-country_US',
       'occupation_Other', 'race_White', 'relationship_Non-Husband',
       'sex_Male', 'workclass_Private'], dtype=object)

In [6]:
model = MLModelCatalog(dataset, 'linear', 'pytorch', cache=True, models_home='/DATA1/srinivas/CARLA/model_cache',\
                      load_online=False)
train_kwargs = {'learning_rate':2e-3, 'epochs':10, 'force_train':False, 'batch_size':256, 'hidden_size': [10,5,10]}

In [7]:
model.train(**train_kwargs)

Loaded model from /DATA1/srinivas/CARLA/model_cache/adult/linear.pt
test accuracy for model: 0.8307667103538663


In [8]:
model.get_ordered_features(dataset.df_test.iloc[0:1])

,age,fnlwgt,education-num,capital-gain,hours-per-week,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.315068,0.230799,0.333333,0.0,0.346939,...,0.0,1.0,1.0,0.0,1.0


In [9]:
model.get_mutable_mask()

array([False, False,  True,  True,  True,  True,  True, False,  True,
       False, False, False,  True])

In [10]:
factual = model.get_ordered_features(dataset.df_test.iloc[0:1])
display(factual)

,age,fnlwgt,education-num,capital-gain,hours-per-week,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.315068,0.230799,0.333333,0.0,0.346939,...,0.0,1.0,1.0,0.0,1.0


In [126]:
immutables = dataset.immutables
display(immutables)
display(model.data.categorical)

['age', 'sex', 'fnlwgt', 'native-country', 'race', 'relationship']

['marital-status',
 'native-country',
 'occupation',
 'race',
 'relationship',
 'sex',
 'workclass']

In [23]:
from carla.models.negative_instances import predict_negative_instances
factuals = predict_negative_instances(model, model.data.df_test)
factuals = model.get_ordered_features(factuals.iloc[:10])

In [24]:
factuals

,age,fnlwgt,education-num,capital-gain,hours-per-week,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.315068,0.230799,0.333333,0.000000,0.346939,...,0.0,1.0,1.0,0.0,1.0
1,0.575342,0.141133,0.933333,0.278283,0.602041,...,1.0,1.0,1.0,1.0,1.0
2,0.328767,0.232481,0.800000,0.000000,0.346939,...,1.0,1.0,1.0,1.0,0.0
3,0.506849,0.229668,0.600000,0.000000,0.346939,...,0.0,1.0,1.0,0.0,1.0
4,0.671233,0.079284,0.800000,0.000000,0.051020,...,1.0,1.0,0.0,1.0,1.0
5,0.315068,0.210745,0.600000,0.000000,0.397959,...,1.0,1.0,0.0,1.0,1.0
6,0.287671,0.160173,0.733333,0.031030,0.397959,...,1.0,1.0,0.0,1.0,1.0
7,0.027397,0.161376,0.600000,0.000000,0.397959,...,0.0,1.0,1.0,1.0,1.0
8,0.273973,0.108523,0.600000,0.000000,0.500000,...,1.0,0.0,0.0,1.0,1.0
9,0.465753,0.033140,0.533333,0.000000,0.397959,...,1.0,1.0,0.0,1.0,1.0


In [25]:
factuals.reset_index(drop=True)

,age,fnlwgt,education-num,capital-gain,hours-per-week,...,occupation_Other,race_White,relationship_Non-Husband,sex_Male,workclass_Private
0,0.315068,0.230799,0.333333,0.000000,0.346939,...,0.0,1.0,1.0,0.0,1.0
1,0.575342,0.141133,0.933333,0.278283,0.602041,...,1.0,1.0,1.0,1.0,1.0
2,0.328767,0.232481,0.800000,0.000000,0.346939,...,1.0,1.0,1.0,1.0,0.0
3,0.506849,0.229668,0.600000,0.000000,0.346939,...,0.0,1.0,1.0,0.0,1.0
4,0.671233,0.079284,0.800000,0.000000,0.051020,...,1.0,1.0,0.0,1.0,1.0
5,0.315068,0.210745,0.600000,0.000000,0.397959,...,1.0,1.0,0.0,1.0,1.0
6,0.287671,0.160173,0.733333,0.031030,0.397959,...,1.0,1.0,0.0,1.0,1.0
7,0.027397,0.161376,0.600000,0.000000,0.397959,...,0.0,1.0,1.0,1.0,1.0
8,0.273973,0.108523,0.600000,0.000000,0.500000,...,1.0,0.0,0.0,1.0,1.0
9,0.465753,0.033140,0.533333,0.000000,0.397959,...,1.0,1.0,0.0,1.0,1.0


In [26]:
gmsc = OnlineCatalog('give_me_some_credit')

In [32]:
gmsc.df.iloc[1].to_numpy()

array([8.53670739e-04, 2.32876712e-01, 1.00000000e-01, 6.55222286e-05,
       6.11457286e-02, 8.33333333e-02, 1.00000000e-01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+00])